In [1]:
import tensorflow as tf
import numpy as np

tf.__version__

'1.14.0'

In [2]:
cpus = tf.config.experimental.list_physical_devices('CPU') # get a list of cpus

Splits the CPU into 2 virtual devices

In [3]:
try:
    tf.config.experimental.set_virtual_device_configuration(cpus[0],
                                                            [tf.config.experimental.VirtualDeviceConfiguration(),
                                                             tf.config.experimental.VirtualDeviceConfiguration()])
    logical_cpus = tf.config.experimental.list_logical_devices('CPU')
except RuntimeError as e:
    print(e)

In [4]:
logical_cpus

[LogicalDevice(name='/job:localhost/replica:0/task:0/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/job:localhost/replica:0/task:0/device:CPU:1', device_type='CPU')]

In [5]:
def generator():
    x = 0
    while True:
        yield np.arange(x, x + 8).reshape((8,1)).astype('float32')
        x += 8

In [6]:
dataset = tf.data.Dataset.from_generator(generator, output_types=tf.float32, output_shapes=(None,1))

W1230 10:07:47.160066 34440 deprecation.py:323] From f:\anaconda3\envs\tensorflow1.14\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:494: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [7]:
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/cpu:0", "/cpu:1"])
dist_dataset = mirrored_strategy.experimental_distribute_dataset(dataset)

W1230 10:07:47.224928 34440 cross_device_ops.py:1177] Not all devices in `tf.distribute.Strategy` are visible to TensorFlow.


In [8]:
with mirrored_strategy.scope():
    iterator = dist_dataset.make_initializable_iterator()
    iterator_init = iterator.initialize()
    next_element = iterator.get_next()

In [9]:
def fn(inputs):
    '''
    inputs: "per-replica" values, such as those produced by a "distributed Dataset"
    '''
    return inputs


with mirrored_strategy.scope():
    per_example_inputs = mirrored_strategy.experimental_run_v2(fn, args=(next_element,))

In [13]:
with mirrored_strategy.scope():
    with tf.Session() as sess:
        sess.run(iterator_init)
        print(sess.run(per_example_inputs.values))

(array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.],
       [6.],
       [7.]], dtype=float32), array([[ 8.],
       [ 9.],
       [10.],
       [11.],
       [12.],
       [13.],
       [14.],
       [15.]], dtype=float32))
